# 要約 
このJupyter Notebookは、Kaggleの「LLM 20 Questions」コンペティションのためのベースライン提出物を作成するためのスターターノートブックです。主な目的は、Pythonパッケージ「rigging」を使用して、LLM（大規模言語モデル）を効率的にテーマに基づく質問応答のゲーム（20の質問）に活用することです。

### 取り組んでいる問題
ノートブックでは、参加者が20の質問ゲームを戦略的にプレイするためのLLMエージェントを作成するためのフレームワークの設定と実装が行われています。このゲームは、限られた質問数で秘密のアイテムを特定しようとするもので、質問を生成する「質問者LLM」と「はい/いいえ」で回答する「回答者LLM」の協力が不可欠です。ノートブックでは、これらのエージェントを構築するためのツールや方法を示しています。

### 使用している手法やライブラリ
1. **rigging**：LLMとのインタラクションを簡素化するフレームワークで、異なるLLMバックエンドの交換や、LLMクエリの検証・再試行が可能です。
2. **vLLM**：モデルをローカル環境でホスティングするためのサーバーで、特定のLLMモデルを読み込むために使用されます。
3. **Hugging Face**：`snapshot_download`関数を利用して、モデルの重みをダウンロードします。
4. **Pydantic**：出力のバリデーションとシリアライズを提供し、データモデルを定義するために使用されます。

ノートブックでは、エージェントのセットアップ、モデルの重みのダウンロード、質問・回答の生成、回答のバリデーションといった様々なプロセスを示し、さらに最終提出物としてPythonスクリプトを作成し、モデルデータを圧縮して提出する際の手順も説明しています。

このノートブックは参加者にさまざまな実践的な手法やツールを提供し、効率的な情報収集と演繹的推論が求められる競技において、大規模言語モデルの適用を促進するための基盤を築くことを目指しています。

---


# 用語概説 
以下は、初心者がつまずく可能性がある、ノートブックに関連する専門用語の解説です。

1. **リギング（Rigging）**:
   - 専用の軽量なフレームワークで、さまざまなLLM（大規模言語モデル）とのインタラクションを管理するために使用されます。LLMをプロダクション環境で使う際のパイプラインを作成することを容易にする目的があります。

2. **Pydantic**:
   - Pythonのデータバリデーションライブラリで、データモデルを定義するのに使います。型ヒントとともにモデルを定義することで、データの整合性を保つための機能を提供します。

3. **vLLM**:
   - LLMをローカルなサービスとしてホスティングするためのツールです。大規模な言語モデルを効率的に実行するために設計されています。

4. **ストップシーケンス（Stop Sequence）**:
   - LLMが生成するテキストにおいて、生成を停止する特定のトークンやシーケンスのことです。これを指定することで、不要な生成結果を防ぐことができます。

5. **アクティベーション対応重み量子化（Activation-Weighted Quantization）**:
   - モデルの重みを圧縮する方法の一つで、出力の精度を維持しつつ、モデルのサイズを小さくするために用いられます。具体的には、重みを整数に置き換え、より少ないビット数で表現します。

6. **ジョッキング（Forking）**:
   - プロセスのコピーを作成する操作です。特定の状態から新しい質問やプロンプトを生成するために使用されています。

7. **XML例（XML Examples）**:
   - 特定のフォーマットや構造に従ってデータを表現する方法。ここではLLMに期待される出力の形式を示すために使用されます。

8. **非同期（Asynchronous）**:
   - プログラムの実行において、ある処理が完了するのを待たずに次の処理を開始できる方式です。特にI/Oを待つような遅延操作を効率的に扱うために重要です。

9. **バリデーター（Validator）**:
   - データの整合性を確認するための一連のルールや関数です。Pydanticにおける `field_validator` は、特定のフィールドが適切な形式や値を持つことを保証します。

10. **エージェント（Agent）**:
    - LLMが特定のタスクを遂行するために設計されたインスタンスやボットのことです。本ノートブックでは質問者と回答者のエージェントがそれぞれ自由に質問や回答を生成します。

これらの用語は、このコンペティションやノートブックの内容に密接に関連しています。初心者はこれらのコンセプトをよく理解することで、ノートブックの理解を深めることができるでしょう。

---


# LLM 20 Questions スターターとリギング

このスターターノートブックでは、Pythonパッケージ「rigging」を使用して、コンペティション用のベースライン提出を作成する方法を示します。このセットアップでは、`llama3`量子化モデルをvLLMを使用して利用します。

## 更新 **2024年6月10日**
- rigging 2.0に対応するようにコードを更新しました。
- 知っているキーワードを活用する非LLM質問エージェントを含めています。ただし、これはプライベートリーダーボードではうまく機能しません。回答エージェントは、riggingを介してLLMを使用します。

## リギングとは何か？

リギングは、Pydantic XMLに基づいた軽量のLLMインタラクションフレームワークです。目的は、プロダクションパイプラインでLLMを利用することをできるだけ簡単かつ効果的にすることです。リギングは、20の質問タスクに最適であり、以下のことができます：
1. 異なるバックエンドLLMモデルの交換を簡単に扱うことができる。
2. 期待される出力を確認し、成功するまで再試行するLLMクエリパイプラインを設計できる。
3. 型ヒント、非同期サポート、pydanticバリデーション、シリアライズなど、モダンなPythonを使用する。

リポジトリをここでスターしてください: https://github.com/dreadnode/rigging
ドキュメントをこちらで読む: https://rigging.dreadnode.io/

リギングは[dreadnode](https://www.dreadnode.io/)によって構築され、維持されています。私たちは日々の作業でリギングを使用しています。

リギングパイプラインの例は次のようになります：
```{python}
chat = rg.get_generator('gpt-4o') \
    .chat(f"南アメリカのAで始まるすべての国の名前を教えてください {Answer.xml_tags()} タグ。") \
    .until_parsed_as(Answer) \
    .run() 
```

生成器は、APIキーが環境変数として保存されていれば、ほとんどの主要なLLM APIでシームレスに作成できます。
```
export OPENAI_API_KEY=...
export TOGETHER_API_KEY=...
export TOGETHERAI_API_KEY=...
export MISTRAL_API_KEY=...
export ANTHROPIC_API_KEY=...
```

このコンペティションでは、モデルをローカルで実行する必要がありますが、幸運にもリギングはバックエンドでtransformersを使用してモデルを実行するサポートをしています。

# セットアップ

以下は、このノートブックのためのセットアップの一部です。ここでは：
- Hugging FaceとKaggleのための秘密のトークンをロードします（オプション）。
- 必要なパッケージをインストールします。
- vLLMサーバーをテストするためのヘルパーユーティリティスクリプトを作成します。

このノートブックは、Kaggleのシークレットを使用したいくつかの隠れたトークンを利用しています。これはオプションであり、コードを実行するために必要ではありません。

In [ ]:
from kaggle_secrets import UserSecretsClient
secrets = UserSecretsClient()

# Hugging Faceトークンを格納するための変数を定義します。
HF_TOKEN: str | None  = None
# KaggleのAPIキーを格納するための変数を定義します。
KAGGLE_KEY: str | None = None
# Kaggleのユーザー名を格納するための変数を定義します。
KAGGLE_USERNAME: str | None = None
    
try:
    # KaggleのシークレットからHugging Faceトークンを取得します。
    HF_TOKEN = secrets.get_secret("HF_TOKEN")
    # KaggleのシークレットからKaggle APIキーを取得します。
    KAGGLE_KEY = secrets.get_secret("KAGGLE_KEY")
    # KaggleのシークレットからKaggleユーザー名を取得します。
    KAGGLE_USERNAME = secrets.get_secret("KAGGLE_USERNAME")
except:
    # 例外が発生した場合は何もしません。
    pass

## パッケージインストール
以下のパッケージをインストールします：
- [rigging](https://github.com/dreadnode/rigging) コンペティション用のLLMパイプラインを作成するために使用します。
- [vLLM](https://github.com/vllm-project/vllm) モデルをローカルで独立したサービスとしてホスティングするために使用します。

また、[uv](https://github.com/astral-sh/uv)を使用します。これにより、これらのパッケージをはるかに早くインストールできます。

**注意:** これらのパッケージは、`/kaggle/tmp/lib`ディレクトリにインストールしています。これは、コンペティションのセットアップの目的でこのパスからファイルを後で提出用のzipファイルに含める必要があるためです。また、vllmの依存関係も`/kaggle/tmp/srvlib`にインストールします。

In [ ]:
# 依存関係のインストール（高速化のためのuv）
!pip install uv==0.1.45

# riggingとkaggleパッケージを指定したディレクトリにインストールします。
!uv pip install -U \
    --python $(which python) \
    --target /kaggle/tmp/lib \
    rigging==2.0.0 \
    kaggle

# vllm、numpyパッケージを別の指定したディレクトリにインストールします。
!uv pip install -U \
    --python $(which python) \
    --target /kaggle/tmp/srvlib \
    vllm==0.4.2 \
    numpy==1.26.4

# LLMをローカルにダウンロード

このコンペティションでは、モデルの重みと共にコードを提出する必要があるため、まずはHugging Faceの`snapshot_download`を使用してモデルの重みをダウンロードします。

ダウンロードするモデルは`solidrust/Meta-Llama-3-8B-Instruct-hf-AWQ`です。これは、コンペティションの要件を満たすサイズのアクティベーション対応重み量子化版のモデルです。

**注意**: 通常の状況でリギングを使用する際にはこのステップは必要ありませんが、コンペティションの提出用zipに含めるために重みを別々にダウンロードしています。

In [ ]:
# モデルをダウンロードする

from huggingface_hub import snapshot_download
from pathlib import Path
import shutil

# モデルを保存するためのパスを設定します。
g_model_path = Path("/kaggle/tmp/model")
# 既にモデルのパスが存在する場合は、そのディレクトリを削除します。
if g_model_path.exists():
    shutil.rmtree(g_model_path)
# モデルの保存先ディレクトリを作成します。
g_model_path.mkdir(parents=True)

# Hugging Faceからモデルのスナップショットをダウンロードします。
snapshot_download(
    repo_id="solidrust/Meta-Llama-3-8B-Instruct-hf-AWQ",  # ダウンロードするモデルのリポジトリID
    ignore_patterns="original*",  # 無視するパターンを指定します。
    local_dir=g_model_path,  # モデルを保存するローカルディレクトリ
    local_dir_use_symlinks=False,  # シンボリックリンクを使用しない設定
    token=globals().get("HF_TOKEN", None)  # Hugging Faceトークンを取得します。
)

モデルの重みは、`/kaggle/tmp/model/`に保存されています。

In [ ]:
!ls -l /kaggle/tmp/model  # /kaggle/tmp/modelディレクトリ内のファイルとその詳細をリスト表示します。

# ヘルパーユーティリティファイル

これらは、vLLMサーバーを起動するために使用するヘルパー関数です。

In [ ]:
%%writefile util.py

# vLLMサーバーを起動するためのヘルパー関数

import subprocess
import os
import socket
import time

# 指定したポートが開いているかどうかを確認する関数
def check_port(port: int) -> bool:
    try:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
            sock.settimeout(1)  # タイムアウトを1秒に設定
            result = sock.connect_ex(('localhost', port))  # ポートに接続を試みる
            if result == 0:  # 接続成功した場合
                return True
    except socket.error:
        pass
    
    return False  # 接続失敗の場合はFalseを返す

# プロセスを実行し、指定したポートが開くまで待機する関数
def run_and_wait_for_port(
    cmd: list[str],  # 実行するコマンド
    port: int,  # 確認するポート番号
    env: dict[str, str] | None,  # 環境変数
    timeout: int = 60,  # タイムアウトの秒数（デフォルトは60秒）
    debug: bool = False  # デバッグモードのフラグ
) -> subprocess.Popen:
    
    # ポートが既に開いている場合はエラーを発生させる
    if check_port(port):
        raise ValueError(f"ポート {port} はすでに開いています")
        
    popen = subprocess.Popen(
        cmd,
        env={**os.environ, **(env or {})},  # 環境変数を統合
        stdout=subprocess.DEVNULL if not debug else None,  # デバッグオフの場合は出力を無効化
        stderr=subprocess.DEVNULL if not debug else None,  # デバッグオフの場合はエラー出力を無効化
    )
    
    start_time = time.time()  # 開始時間を記録
    while time.time() - start_time < timeout:  # タイムアウトまでループ
        if check_port(port):  # ポートが開いたか確認
            return popen  # ポートが開いた場合、プロセスを返す
        time.sleep(1)  # 1秒待機
    
    popen.terminate()  # タイムアウトした場合はプロセスを終了
    raise Exception(f"プロセスは {timeout} 秒以内にポート {port} を開きませんでした。")

# vLLMサーバーを起動してテスト

私たちのモデルはvLLMサーバーを使用してホスティングされます。以下では、このノートブックを起動して、Kaggle環境での動作を理解します。

In [ ]:
# vLLMのパスと設定

import importlib
from pathlib import Path
import util

# ユーティリティモジュールを再読み込みします
util = importlib.reload(util)

# サーバーライブラリのパスを設定します
g_srvlib_path = Path("/kaggle/tmp/srvlib")
assert g_srvlib_path.exists()  # パスが存在することを確認します

# モデルのパスを設定します
g_model_path = Path("/kaggle/tmp/model")
assert g_model_path.exists()  # パスが存在することを確認します

# vLLMサーバーのポート番号とモデル名を設定します
g_vllm_port = 9999
g_vllm_model_name = "custom"

In [ ]:
# サブプロセスを使用してvLLMサーバーを実行します
vllm = util.run_and_wait_for_port([
    "python", "-m",  # Pythonモジュールとして実行
    "vllm.entrypoints.openai.api_server",  # vLLMのAPIサーバーを起動
    "--enforce-eager",  # イagerを強制するオプション
    "--model", str(g_model_path),  # モデルのパスを指定
    "--port", str(g_vllm_port),  # サーバーがリッスンするポート番号を指定
    "--served-model-name", g_vllm_model_name  # 提供するモデルの名前を指定
],
    g_vllm_port,  # 確認するポート番号
    {"PYTHONPATH": str(g_srvlib_path)},  # 環境変数にサーバーライブラリのパスを設定
    debug=False  # デバッグモードをオフにします
)

print("vLLMが起動しました")

llama3モデルが最初のTesla T4 GPUにロードされていることが確認できます。

In [ ]:
!nvidia-smi  # NVIDIA GPUの状態を表示します。モデルが正しくロードされているか確認するために必要です。

## モデルの検証

最初のリギングジェネレーターを作成しましょう。リギングでは、ジェネレーターが強力なLLMパイプラインを作成するための基盤となります。

In [ ]:
# リギングとの接続

import sys
import logging

# ライブラリのパスを追加
sys.path.insert(0, "/kaggle/tmp/lib")

logging.getLogger("LiteLLM").setLevel(logging.WARNING)  # ログレベルをWARNINGに設定

import rigging as rg

# リギングジェネレーターを取得します
generator = rg.get_generator(
    f"openai/{g_vllm_model_name},"  # モデルの名前を指定します
    f"api_base=http://localhost:{g_vllm_port}/v1,"  # APIのベースURL
    "api_key=sk-1234,"  # APIキー（例示）
    "stop=<|eot_id|>"  # Llama用のストップシーケンスを指定
)

# ジェネレーターを使用してチャットを行います
answer = await generator.chat("Say Hello!").run()

print()
print('[リギングチャット]')
print(type(answer), answer)  # 答えのタイプと内容を表示

print()
print('[LLMの応答のみ]')
print(type(answer.last), answer.last)  # LLMの最後の応答を表示

print()
answer_string = answer.last.content  # 応答の内容を取得
print('[LLMの応答を文字列として]')
print(answer.last.content)  # 応答の内容を表示

## 結果をpandasデータフレームに変換する

`to_df()`メソッドを使用することで、チャット履歴を簡単にpandasデータフレームに変換できます。

In [ ]:
answer.to_df()  # チャットの応答をpandasデータフレームに変換して表示します。

## モデルパラメータの変更

データベース接続文字列と同様に、リギングジェネレーターは、使用するプロバイダー、モデル、APIキー、生成パラメータなどを定義する文字列として表現できます。形式は以下の通りです：

```
<provider>!<model>,<**kwargs>
```

例えば、ここでは追加のパラメータを使用してモデルをロードします：
- temperature=0.9
- max_tokens=512

これらの詳細については、ドキュメントをこちらでご覧ください: https://rigging.dreadnode.io/topics/generators/#overload-generation-params

In [ ]:
# パラメータを変更してリギングジェネレーターを取得します
generator = rg.get_generator(
    f"openai/{g_vllm_model_name},"  # モデルの名前を指定します
    f"api_base=http://localhost:{g_vllm_port}/v1,"  # APIのベースURL
    "api_key=sk-1234,"  # APIキー（例示）
    "temperature=0.9,max_tokens=512,"  # 生成パラメータを指定
    "stop=<|eot_id|>"  # Llama用のストップシーケンスを指定
)

別の方法として、`rg.GenerateParams`クラスを使用してこれらのパラメータを設定することもできます。このクラスを使用すると、さまざまなモデルパラメータを設定できます：

```
rg.GenerateParams(
    *,
    temperature: float | None = None,
    max_tokens: int | None = None,
    top_k: int | None = None,
    top_p: float | None = None,
    stop: list[str] | None = None,
    presence_penalty: float | None = None,
    frequency_penalty: float | None = None,
    api_base: str | None = None,
    timeout: int | None = None,
    seed: int | None = None,
    extra: dict[str, typing.Any] = None,
)
```

詳細については、こちらのドキュメントをご覧ください: https://rigging.dreadnode.io/api/generator/#rigging.generator.GenerateParams

In [ ]:
# GenerateParamsを使用してパラメータを設定
rg_params = rg.GenerateParams(
    temperature=0.9,  # 温度を0.9に設定
    max_tokens=512,  # 最大トークン数を512に設定
)

# ジェネレーターを使用してチャットを作成
base_chat = generator.chat(params=rg_params)

# 質問「How is it going?」を投げて応答を取得
answer = await base_chat.fork('How is it going?').run()
print(answer.last.content)  # LLMの応答を表示

また、チェーン内で`params`を使用してパラメータを設定することもできます。

In [ ]:
# パラメータを設定せずにチャットを作成
base_chat = generator.chat()  # パラメータは設定しない

# 質問「How is it going?」を投げて応答を取得（この時にパラメータを指定）
answer = await base_chat.fork('How is it going?') \
    .with_(temperature=0.9, max_tokens=512) \  # 温度を0.9、最大トークン数を512に設定
    .run()

print(answer.last.content)  # LLMの応答を表示

# パースされた出力の例

次に、以下のパイプラインを作成します：
1. `Answer`という名前のリギングモデルを作成します。これは、モデルの結果からパースすることが期待される出力を説明します。
    - ここに一部のバリデーターを追加して、出力が`yes`または`no`のいずれかであることを保証します。
    - これは完全にカスタマイズ可能です。
    - ここでは、`validate_content`が応答が期待される出力（小文字で「yes」または「no」で始まる）に準拠していることを確認します。
2. プロンプト内で`Answer.xml_example()`を使用して、LLMに出力がどのように見えるべきかを知らせることができます。
3. 後で`.until_parsed_as(Answer)`を使用して、LLMの出力がここで定義した通りに抽出されることを確認します。

**注意**: `until_parsed_as()`にはデフォルトで5である`max_rounds`パラメータを指定することができます。

In [ ]:
import typing as t
from pydantic import field_validator

# モデルクラスAnswerを定義します
class Answer(rg.Model):
    content: t.Literal["yes", "no"]  # 出力が"yes"または"no"であることを指定

    # contentフィールドのバリデーションを定義します
    @field_validator("content", mode="before")
    def validate_content(cls, v: str) -> str:
        for valid in ["yes", "no"]:  # 有効な値をリストで定義
            if v.lower().startswith(valid):  # 値が小文字で"yes"または"no"で始まるか確認
                return valid  # 有効な場合はその値を返す
        raise ValueError("無効な回答です。'yes'または'no'である必要があります")  # 無効な場合はエラーを発生させる

    # XML形式の例を返すクラスメソッドを定義します
    @classmethod
    def xml_example(cls) -> str:
        return f"{Answer.xml_start_tag()}**yes/no**{Answer.xml_end_tag()}"  # XMLタグで囲まれた例を返す

In [ ]:
# XML例がどのように見えるかを確認します。これはプロンプトに使用できます。
Answer.xml_example()  # AnswerモデルのXML例を表示します。

In [ ]:
# ジェネレーターを再取得し、プロンプトを設定します
generator = rg.get_generator(
    f"openai/{g_vllm_model_name},"  # モデルの名前を指定
    f"api_base=http://localhost:{g_vllm_port}/v1,"  # APIのベースURL
    "api_key=sk-1234,"  # APIキー（例示）
    "stop=<|eot_id|>"  # Llama用のストップシーケンスを指定
)

keyword = 'Tom Hanks'  # 秘密の単語
category = 'Famous Person'  # カテゴリ
last_question = 'Is it a famous person?'  # 最後の質問

# プロンプトを設定します
prompt = f"""\
            このゲームの秘密の単語は "{keyword}" [{category}] です。

            あなたは現在、上記の単語についての質問に答えています。

            次の質問は "{last_question}" です。

            上記のはい/いいえの質問に答え、以下の形式で回答してください：
            {Answer.xml_example()}

            - あなたの回答は、上記のキーワードに基づいて正確であるべきです
            - 常に「yes」または「no」で答えてください

            答えは何ですか？
"""

# チャットを実行して応答を取得します
chat = await (
    generator
    .chat(prompt)  # プロンプトを指定
    .until_parsed_as(Answer, max_rounds=50)  # 50ラウンドまでパースを試みる
    .run()
)

print('=== 完全なチャット ===')
print(chat)  # チャットの全体を表示

print()
print('=== LLMの応答のみ ===')
print(chat.last)  # LLMの最後の応答を表示

print()
print('=== パースされた回答 ===')
print(chat.last.parse(Answer).content)  # パースされた応答の内容を表示

# リギングを使った質問者チャットパイプラインの作成

次に、キーワードが何であるかを特定する手助けをする質問者パイプラインを作成します。

まず、出力をパースするために使用する`Question`オブジェクトを作成します。

In [ ]:
from pydantic import StringConstraints  # noqa

# 空白を取り除くための型定義
str_strip = t.Annotated[str, StringConstraints(strip_whitespace=True)]

# Questionモデルクラスを定義
class Question(rg.Model):
    content: str_strip  # contentフィールドに空白を取り除いた文字列を設定

    # XML形式の例を返すクラスメソッドを定義
    @classmethod
    def xml_example(cls) -> str:
        return Question(content="**question**").to_pretty_xml()  # 例となる質問をXML形式で返す

In [ ]:
# 基本チャットを作成し、質問を行います
base = generator.chat("""\
あなたは「20の質問」ゲームの才能あるプレイヤーです。あなたは正確で、集中力があり、構造的なアプローチを取ります。役に立つ質問を作成し、推測を行い、キーワードに関する質問に答えます。

""")


# 質問を行うためのチャットを構築します
question_chat = await (base.fork(
    f"""\
    あなたは現在、次の質問をしています。

    質問を作成し、以下の形式で回答してください：
    {Question.xml_example()}

    - あなたの応答は、最も多くの情報を集めるための集中した質問であるべきです
    - 質問は一般的なところから始めてください
    - 残りの検索空間を二分しようと常に努めてください
    - 前の質問と回答に注意を払ってください

    次の質問は何ですか？
    """
)
.until_parsed_as(Question, attempt_recovery=True)  # 質問のパースを試みる
.run()
)

In [ ]:
# 会話のデータフレーム表現を表示します
question_chat.to_df()  # 質問チャットの内容をpandasデータフレームに変換して表示します。

現在、LLMの応答には質問が含まれており、以下のように質問をパースできると確信しています：

In [ ]:
# LLMの応答から質問をパースします
question = question_chat.last.parse(Question).content  # 最後の応答をQuestionモデルでパースし、内容を取得
print(question)  # 取得した質問を表示

# キーワードデータフレームを作成する
**注意**: これは公開セットの可能なキーワードを知っているためにのみ機能します。これが最終リーダーボードでは機能しないことに注意してください。

In [ ]:
# キーワードのリストを取得するためにファイルをダウンロードします
!wget -O keywords_local.py https://raw.githubusercontent.com/Kaggle/kaggle-environments/master/kaggle_environments/envs/llm_20_questions/keywords.py  # キーワードのPythonファイルを指定した名前でダウンロードします。

In [ ]:
# ダウンロードしたキーワードファイルの先頭部分を表示します
!head keywords_local.py  # keywords_local.pyファイルの最初の数行を表示して内容を確認します。

In [ ]:
import sys
import json
import pandas as pd
sys.path.append('./')  # 現在のディレクトリをパスに追加
from keywords_local import KEYWORDS_JSON  # ダウンロードしたキーワードデータをインポート

# テキストの最初の単語を大文字にし、残りを小文字にする関数
def capitalize_first_word(text):
    if not text:
        return text  # テキストが空の場合はそのまま返す
    return text[0].upper() + text[1:].lower()  # 最初の文字を大文字にし、それ以外を小文字にする

# キーワードデータフレームを作成する関数
def create_keyword_df(KEYWORDS_JSON):
    # JSON形式のキーワードデータを読み込む
    keywords_dict = json.loads(KEYWORDS_JSON)

    category_words_dict = {}
    all_words = []
    all_cat_words = []
    # 各カテゴリの単語を辞書に格納
    for d in keywords_dict:
        words = [w['keyword'] for w in d['words']]  # 各単語を抽出
        cat_word = [(d['category'], w['keyword']) for w in d['words']]  # カテゴリと単語のタプルを作成
        category_words_dict[d['category']] = words  # カテゴリ毎に単語リストを保持
        all_words += words  # すべての単語を集約
        all_cat_words += cat_word  # カテゴリと単語のタプルを集約

    # データフレームを作成
    keyword_df = pd.DataFrame(all_cat_words, columns=['category','keyword'])
    keyword_df['first_letter'] = keyword_df['keyword'].str[0]  # 単語の最初の文字を抽出
    keyword_df['second_letter'] = keyword_df['keyword'].str[1]  # 単語の第二の文字を抽出
    keyword_df.to_parquet('keywords.parquet')  # データフレームをparquetファイルとして保存
    
# キーワードデータフレームを作成する関数を呼び出し
create_keyword_df(KEYWORDS_JSON)

In [ ]:
# 保存したキーワードデータフレームを読み込み、サンプルを表示します
keywords_df = pd.read_parquet('keywords.parquet')  # parquetファイルからデータフレームを読み込む
keywords_df.sample(10)  # データフレームからランダムに10行をサンプル表示

In [ ]:
# 各カテゴリの出現回数をカウントして表示します
keywords_df['category'].value_counts()  # カテゴリ毎の出現頻度を集計し表示

# 最終提出用の`main.py`スクリプトを作成する

私たちの最終提出物は、`main`ファイルを含む圧縮ディレクトリになります。以下にそのファイルの内容を示します。

In [ ]:
%%writefile main.py

# メインエージェントファイル

import itertools
import os
import sys
import typing as t
from pathlib import Path
import logging

import string
import numpy as np
import pandas as pd

# パスの修正

g_working_path = Path('/kaggle/working')
g_input_path = Path('/kaggle/input')
g_temp_path = Path("/kaggle/tmp")
g_agent_path = Path("/kaggle_simulations/agent/")

g_model_path = g_temp_path / "model"
g_srvlib_path = g_temp_path / "srvlib"
g_lib_path = g_temp_path / "lib"

if g_agent_path.exists():
    g_lib_path = g_agent_path / "lib"
    g_model_path = g_agent_path / "model"
    g_srvlib_path = g_agent_path / "srvlib"
else:
    g_agent_path = Path('/kaggle/working')
    
sys.path.insert(0, str(g_lib_path))

# ロギングのノイズを削減

logging.getLogger("LiteLLM").setLevel(logging.WARNING)

# 固定インポート

import util # noqa
import rigging as rg  # noqa
from pydantic import BaseModel, field_validator, StringConstraints  # noqa

# 定数

g_vllm_port = 9999
g_vllm_model_name = "custom"

g_generator_id = (
    f"openai/{g_vllm_model_name}," \
    f"api_base=http://localhost:{g_vllm_port}/v1," \
    "api_key=sk-1234," \
    "stop=<|eot_id|>" # Llama requires some hand holding
)

# タイプ

str_strip = t.Annotated[str, StringConstraints(strip_whitespace=True)]

class Observation(BaseModel):
    step: int
    role: t.Literal["guesser", "answerer"]
    turnType: t.Literal["ask", "answer", "guess"]
    keyword: str
    category: str
    questions: list[str]
    answers: list[str]
    guesses: list[str]
    
    @property
    def empty(self) -> bool:
        return all(len(t) == 0 for t in [self.questions, self.answers, self.guesses])
    
    def get_history(self) -> t.Iterator[tuple[str, str, str]]:
        return itertools.zip_longest(self.questions, self.answers, self.guesses, fillvalue="[none]")

    def get_history_as_xml(self, *, skip_guesses: bool = False) -> str:
        if not self.empty:
            history = "\n".join(
            f"""\
            <turn-{i}>
            Question: {question}
            Answer: {answer}
            {'Guess: ' + guess if not skip_guesses else ''}
            </turn-{i}>
            """
            for i, (question, answer, guess) in enumerate(self.get_history())
            )
            return history
        return "none yet."


class Answer(rg.Model):
    content: t.Literal["yes", "no"]

    @field_validator("content", mode="before")
    def validate_content(cls, v: str) -> str:
        for valid in ["yes", "no"]:
            if v.lower().startswith(valid):
                return valid
        raise ValueError("無効な回答です。'yes'または'no'である必要があります")

    @classmethod
    def xml_example(cls) -> str:
        return f"{Answer.xml_start_tag()}yes/no{Answer.xml_end_tag()}"


class Question(rg.Model):
    content: str_strip

    @classmethod
    def xml_example(cls) -> str:
        return Question(content="質問").to_pretty_xml()


class Guess(rg.Model):
    content: str_strip

    @classmethod
    def xml_example(cls) -> str:
        return Guess(content="物事/場所").to_pretty_xml()


# 関数

async def ask(base: rg.ChatPipeline, observation: Observation) -> str:
    if observation.step == 0:
        # 最初の質問をオーバーライドしてキーワードバグを修正
        return "Are we playing 20 questions?"
    
    try:
        chat = await (
             base.fork(
                f"""\
                あなたは現在、次の質問をしています。

                <game-history>
                {observation.get_history_as_xml(skip_guesses=True)}
                </game-history>

                上記の履歴に基づいて、次の最も有用なはい/いいえの質問をして、以下の形式で回答してください：
                {Question.xml_example()}

                - あなたの回答は、最も多くの情報を集めるための集中した質問であるべきです
                - 質問は一般的なところから始めてください
                - 残りの検索空間を二分しようと常に努めてください
                - 前の質問と回答に注意を払ってください

                次の質問は何ですか？
                """
            )
            .until_parsed_as(Question, attempt_recovery=True, max_rounds=20)
            .run()
        )
        return chat.last.parse(Question).content.strip('*')
    except rg.error.MessagesExhaustedMaxRoundsError:
        return 'Is it a person?'

async def answer(base: rg.ChatPipeline, observation: Observation) -> t.Literal["yes", "no"]:
    if not observation.keyword:
        print("Keyword wasn't provided to answerer", file=sys.stderr)
        return "yes" # override until keyword bug is fixed.
            
    last_question = observation.questions[-1]
    
    try:
        responses = []
        for i in range(5):
            # 5回ループして最も多い応答を取得
            chat = await (
                base.fork(
                    f"""\
                    Provide the best yes/no answer to the question about the keyword [{observation.keyword}] in the category [{observation.category}]

                    [QUESTION] "{last_question}" [/QUESTION]

                    Remember they keyword is [{observation.keyword}]

                    Answer the yes/no question above and place it in the following format:
                    {Answer.xml_example()}
                    """
                )
                .until_parsed_as(Answer, attempt_recovery=True, max_rounds=20)
                .run()
            )
            responses.append(chat.last.parse(Answer).content.strip('*'))
            
        print(f'Responses are {responses}')
        return pd.Series(responses).value_counts().index[0]
    except rg.error.MessagesExhaustedMaxRoundsError:
        print('%%%%%%%%%%%% エラーが発生したため「はい」と答えます %%%%%%%%%%%% ')
        return 'yes'

async def guess(base: rg.ChatPipeline, observation: Observation) -> str:
    try:
        chat = await (
            base.fork(
                f"""\
                あなたは現在、キーワードの情報に基づいて推測を行っています。

                <game-history>
                {observation.get_history_as_xml()}
                </game-history>

                上記の履歴に基づいて、キーワードの次に最良の推測を1つ作成し、以下の形式で回答してください：
                {Guess.xml_example()}

                - 上記の履歴に基づいて繰り返しの推測を避けてください
                - 推測は特定の人物、場所、または物であるべきです

                あなたの推測は何ですか？
                """
            )
            .until_parsed_as(Guess, attempt_recovery=True, max_rounds=20)
            .run()
        )

        return chat.last.parse(Guess).content.strip('*')
    except rg.error.MessagesExhaustedMaxRoundsError:
        return 'france'
    
# vLLMとジェネレーターの設定

try:
    vllm = util.run_and_wait_for_port([
        "python", "-m",
        "vllm.entrypoints.openai.api_server",
        "--enforce-eager",
        "--model", str(g_model_path),
        "--port", str(g_vllm_port),
        "--served-model-name", g_vllm_model_name
    ], g_vllm_port, {"PYTHONPATH": str(g_srvlib_path)})

    print("vLLMが起動しました")
except ValueError:
    print('vLLMはすでに実行中です')
    
    
generator = rg.get_generator(g_generator_id)

base =  generator.chat("""\
あなたは「20の質問」ゲームの才能あるプレイヤーです。あなたは正確で、集中力があり、
構造的なアプローチを取ります。役に立つ質問を作成し、推測を行い、キーワードに関する質問に答えます。

""")

# エントリーポイント
def format_first_letter_question(letters):
    if not letters:
        return "キーワードの最初の文字は何ですか？"
    
    if len(letters) == 1:
        return f"キーワードは'{letters[0]}'で始まりますか？"
    
    formatted_letters = ", ".join(f"'{letter}'" for letter in letters[:-1])
    formatted_letters += f"または'{letters[-1]}'で始まりますか？"
    
    return f"キーワードの最初の文字は次のいずれかですか: {formatted_letters}?"

import re

def extract_letters_from_question(question):
    pattern = r"'([a-zA-Z])'"
    matches = re.findall(pattern, question)
    return matches

# シンプルな質問者エージェント
class SimpleQuestionerAgent():
    def __init__(self, keyword_df: pd.DataFrame):
        self.keyword_df = keyword_df
        self.keyword_df_init = keyword_df.copy()
        self.round = 0
        self.category_questions = [
            "Are we playing 20 questions?",
            "Is the keyword a thing that is not a place?",
            "Is the keyword a place?",
        ]
        self.found_category = False
        
    def filter_keywords(self, obs):
        print(self.keyword_df.shape)
        # 過去の回答に基づいてkeyword_dfをフィルタリングします
        for i, answer in enumerate(obs.answers):
            if obs.questions[i] in self.category_questions:
                if answer == 'yes':
                    if obs.questions[i] == "Is the keyword a thing that is not a place?":
                        self.found_category = 'things'
                    if obs.questions[i] == "Is the keyword a place?":
                        self.found_category = 'place'
                    fc = self.found_category
                    self.keyword_df = self.keyword_df.query('category == @fc').reset_index(drop=True)
    
            if obs.questions[i].startswith('Does the keyword start '):
                if self.keyword_df['first_letter'].nunique() <= 1:
                    break
                letter_question = obs.questions[i]
                letters = extract_letters_from_question(letter_question)
                self.keyword_df = self.keyword_df.reset_index(drop=True).copy()
                if obs.answers[i] == 'yes':
                    self.keyword_df = self.keyword_df.loc[
                        self.keyword_df['first_letter'].isin(letters)].reset_index(drop=True).copy()
                elif obs.answers[i] == 'no':
                    self.keyword_df = self.keyword_df.loc[
                        ~self.keyword_df['first_letter'].isin(letters)].reset_index(drop=True).copy()
        if len(self.keyword_df) == 0:
            # リセット
            self.keyword_df = self.keyword_df_init.copy()
            
    def get_letters(self, obs, max_letters=20):
        n_letters = self.keyword_df['first_letter'].nunique()
        sample_letters = self.keyword_df['first_letter'].drop_duplicates().sample(n_letters // 2).values.tolist()
        sample_letters = sample_letters[:max_letters]
        print('sample letters', n_letters, sample_letters)
        return sample_letters
    
    def __call__(self, obs, *args):
        if len(self.keyword_df) == 0:
            # リセット
            self.keyword_df = self.keyword_df_init.copy()
        self.filter_keywords(obs)
        if obs.turnType == 'ask':
            self.round += 1
            if (self.round <= 3 and not self.found_category):
                response = self.category_questions[self.round - 1]
            else:
                sample_letters = self.get_letters(obs)
                if len(sample_letters) == 0:
                    n_sample = min(len(self.keyword_df), 10)
                    possible_keywords = ", ".join(self.keyword_df['keyword'].sample(n_sample).values.tolist())
                    response = f"以下のいずれかがキーワードですか？ {possible_keywords}"
                else:
                    sample_letters_str = str(sample_letters).replace('[','').replace(']','')
                    response = format_first_letter_question(sample_letters)
        elif obs.turnType == 'guess':
            response = self.keyword_df['keyword'].sample(1).values[0]
            # 推測した単語を除外
            updated_df = self.keyword_df.loc[self.keyword_df['keyword'] != response].reset_index(drop=True).copy()
            if len(updated_df) >= 1:
                self.keyword_df = updated_df.copy()
            else:
                self.keyword_df = self.keyword_df_init.copy() # データフレームをリセット
        return response


keyword_df = pd.read_parquet(f'{g_agent_path}/keywords.parquet')
question_agent = None

async def observe(obs: t.Any) -> str:
    observation = Observation(**obs.__dict__)
    global question_agent
    if question_agent is None:
        question_agent = SimpleQuestionerAgent(keyword_df)

    try:
        match observation.turnType:
            case "ask":
                return question_agent(obs)
            case "answer":
                return await answer(base, observation)
            case "guess":
                return question_agent(obs)

            case _:
                raise ValueError("Unknown turn type")
    except Exception as e:
        print(str(e), file=sys.stderr)
        raise

def agent_fn(obs: t.Any, _: t.Any) -> str:
    # フレームワーク内で実行する際の非同期ゲート
    import asyncio
    return asyncio.run(observe(obs))

# エージェントを自身に対してテストする

In [ ]:
# 最初の文字に関する質問を形式化する関数
def format_first_letter_question(letters):
    if not letters:
        return "キーワードの最初の文字は何ですか？"
    
    if len(letters) == 1:
        return f"キーワードは'{letters[0]}'で始まりますか？"
    
    formatted_letters = ", ".join(f"'{letter}'" for letter in letters[:-1])
    formatted_letters += f"または'{letters[-1]}'で始まりますか？"
    
    return f"キーワードの最初の文字は次のいずれかですか: {formatted_letters}?"

# 例として['a','b','c']を渡して、関数の結果を確認します
format_first_letter_question(['a','b','c'])  

import re

# 質問から文字を抽出する関数
def extract_letters_from_question(question):
    pattern = r"'([a-zA-Z])'"  # 文字列に含まれるシングルクォートで囲まれた文字を探す正規表現
    matches = re.findall(pattern, question)  # 正規表現にマッチしたものをリストとして返す
    return matches  # 抽出した文字のリストを返す

In [ ]:
# 自動再読み込みを設定し、必要なモジュールをインポートします
%load_ext autoreload  # 自動再読み込みを有効にする
%autoreload 2  # すべてのモジュールを変更時に自動で再読み込み
from main import Observation, agent_fn, observe  # mainから必要なクラス・関数をインポート

In [ ]:
# vLLMが実行中かどうかを確認します
!ps -aef | grep vllm  # プロセスのリストからvllmを検索して表示します

In [ ]:
import pandas as pd

# キーワードデータフレームを読み込み、サンプルを抽出します
keyword_df = pd.read_parquet('keywords.parquet')
sample = keyword_df.sample(1)  # ランダムに1つのサンプルを取得

# 観察オブジェクトを作成
obs = Observation(step=0,
    role='guesser',  # ロールを'guesser'に設定
    turnType="ask",  # ターンタイプを'ask'に設定
    keyword=sample['keyword'].values[0],  # サンプルからキーワードを取得
    category=sample['category'].values[0],  # サンプルからカテゴリを取得
    questions=[],  # 質問のリストを初期化
    answers=[],  # 回答のリストを初期化
    guesses=[],  # 推測のリストを初期化
)

# 20回の質問と回答のループ
for i in range(20):
    obs.role = 'guesser'
    obs.turnType = 'ask'
    question = await observe(obs)  # 質問を観察関数で取得
    print(f'[{i} Question]: {question}')  # 質問を表示
    obs.questions.append(question)  # 質問リストに追加
    obs.role = 'answerer'
    obs.turnType = 'answer'
    answer = await observe(obs)  # 回答を観察関数で取得
    obs.answers.append(answer)  # 回答リストに追加
    
    # 正解を設定
    if obs.questions[-1].startswith('Are we playing 20 questions?'):
        gt_answer = answer  # 何でも良い
    elif obs.questions[-1].startswith('Is the keyword a thing that is not a place?'):
        gt_answer = 'yes' if sample['category'].values[0] == 'things' else 'no'
    elif obs.questions[-1].startswith('Is the keyword a place?'):
        gt_answer = 'yes' if sample['category'].values[0] == 'place' else 'no'
    elif obs.questions[-1].startswith('Does the keyword start'):
        letters_guess = extract_letters_from_question(obs.questions[-1])  # 質問から文字を抽出
        gt_answer = 'yes' if obs.keyword[0] in letters_guess else 'no'
    elif obs.questions[-1].startswith('Is the keyword one of the following?'):
        possible_kw = obs.questions[-1].replace('Is the keyword one of the following? ', '').split(',')
        possible_kw = [c.strip(' ') for c in possible_kw]
        print(possible_kw)
        gt_answer = 'yes' if obs.keyword in possible_kw else 'no'

    print(f'[{i} Answer]: {answer} [True Answer]: {gt_answer}')  # 回答と正しい回答を表示
    if answer != gt_answer:
        break  # 回答が異なる場合はループを抜ける

    obs.role = 'guesser'
    obs.turnType = 'guess'
    guess = await observe(obs)  # 推測を観察関数で取得
    print(f'[{i} Guess]: {guess} - [Keyword]: {obs.keyword}')  # 推測とキーワードを表示
    obs.guesses.append(guess)  # 推測リストに追加
    if guess == obs.keyword:
        print('GOT IT!')  # 正しい推測ができた場合のメッセージ
        break
        
    obs.step += 1  # ステップを更新

# モデルとコードを圧縮して提出用に準備する

In [ ]:
# 提出用の圧縮に必要なパッケージをインストールします
!apt install pigz pv  # pigz（並列圧縮用のgzip）とpv（仮想パイプ）をインストールします

In [ ]:
# モデルとコードを圧縮して提出用のtar.gzファイルを作成します
!tar --use-compress-program='pigz --fast' \
    -cf submission.tar.gz \  # 圧縮ファイルの作成
    --dereference \  # シンボリックリンクを解決
    -C /kaggle/tmp model lib srvlib \  # /kaggle/tmp内のモデル、lib、srvlibを追加
    -C /kaggle/working main.py util.py \  # /kaggle/working内のmain.pyとutil.pyを追加
    -C /kaggle/working keywords.parquet  # keywords.parquetを追加

In [ ]:
# 作成したファイルやディレクトリの内容を表示します
!ls -GFlash --color  # 詳細情報を色付きで表示します。

# Kaggle CLIを使用した提出

オプションとして、ノートブックを再実行することなく、Kaggle CLIインターフェースを使用して提出することができます。

In [ ]:
# Kaggle CLIを使用してコンペティションに提出するためのコマンドを示します（コメントアウトされています）
# 環境変数にKaggleのユーザー名とAPIキーを設定し、以下のコマンドを実行します
# !KAGGLE_USERNAME={KAGGLE_USERNAME} \
#  KAGGLE_KEY={KAGGLE_KEY} \
#  kaggle competitions submit -c llm-20-questions -f submission.tar.gz -m "submit from notebook"

# コメント 

> ## Bhanu Prakash M
> 
> Hi [@robikscube](https://www.kaggle.com/robikscube),
> 
> vLLMサーバーを実行する方法を教えてもらえますか？デバッグをtrueに設定した後、以下のエラーが発生します。
> 
> INFO 06-18 21:44:58 selector.py:69] Cannot use FlashAttention-2 backend for Volta and Turing GPUs.
> 
> INFO 06-18 21:44:58 selector.py:32] Using XFormers backend.
> 
> そして長いトレースバックエラーの最終文は
> 
> ValueError: Bfloat16 is only supported on GPUs with compute capability of at least 8.0. Your Tesla P100-PCIE-16GB GPU has compute capability 6.0. You can use float16 instead by explicitly setting the dtype flag in CLI, for example: --dtype=half
> 
> 
> > ## Rob MullaTopic Author
> > 
> どのモデルを実行しようとしていますか？
> 
> 
> 
> > > ## Bhanu Prakash M
> > > phi-3モデルをllamaフォーマットに変換された重みと共に使用しています。
> > > 
> > > [https://huggingface.co/rhysjones/Phi-3-mini-mango-1-llamafied](https://huggingface.co/rhysjones/Phi-3-mini-mango-1-llamafied)
> > > 
> > > これがそのモデルです。
> > > 
> > > 
> > > > ## Bhanu Prakash M
> > > > [@robikscube](https://www.kaggle.com/robikscube) 何か更新はありますか？
> > > >
> > > > ## Rob MullaTopic Author
> > > > ここで動作させました: [https://www.kaggle.com/code/robikscube/phi3-intro-to-rigging-for-llm-20-questions/](https://www.kaggle.com/code/robikscube/phi3-intro-to-rigging-for-llm-20-questions/)
> > > > 
> > > 


---

> ## OminousDude
> 
> "Process did not open port 9999 within 120 seconds"というエラーが出るのはなぜですか？ [@robikscube](https://www.kaggle.com/robikscube)
> 
> 
> > ## Rob MullaTopic Author
> > 
> ちょっと見てみます！教えてくれてありがとう。
> > 
> 
> 
> > > ## OminousDude
> > > ありがとうございます！リギングを使ってコードを高速化しようとしているが、使用しているモデルが時間配分に合わず、このコードが本当に役立っています。
> > > 
> > > 

---

> ## OminousDude
> 
> あなたのコードをテストしてみましたが、実行すると"AttributeError: 'coroutine' object has no attribute 'last'"という例外で失敗しました。このエラーに遭遇したことはありますか？
> 
> 
> > ## Rob MullaTopic Author
> > 
> 教えてくれてありがとう。この問題も確認しています。リギングは現在開発中で、この変更は最新のリリースから来たようです。ノートブックを更新してこの変更を修正するか、問題を解決するはずの古いバージョンのリギングを固定するかもしれません。
> > 
> 
> > > ## OminousDude
> > > わかりました。ありがとう！
> > > 
> > > > ## OminousDude
> > > > バージョン7は動作しますか？
> > > > 
> > > 

---

> ## OminousDude
> 
> 私はこれを自分のローカルマシンで試していますが、動作していません。理由がわかりますか？ [@robikscube](https://www.kaggle.com/robikscube) 
> 
> 

---

> ## OminousDude
> 
> これがバグかどうかわからないが、このコードは「solidrust/Meta-Llama-3-8B-Instruct-hf-AWQ」でしか動作していません。このコードを使ってより大きなバージョンのLlamaを試しているときに気づきました。
> 
> 